# Integrating Deepgram with Dolby.io for Conference Recording Transcription

In [ ]:
# Install dependencies (first time only)
! pip install asyncio deepgram-sdk dolbyio-rest-apis
# asyncio: https://docs.python.org/3/library/asyncio.html
# deepgram-sdk: https://pypi.org/project/deepgram-sdk/
# dolbyio-rest-apis: https://pypi.org/project/dolbyio-rest-apis/

## Step 1. Generate Bearer Token for Communications API

In [ ]:
from dolbyio_rest_apis.communications import authentication
import asyncio

# Input your Dolby.io Communications Credentials here
CONSUMER_KEY = "<DOLBYIO_CONSUMER_KEY>"
CONSUMER_SECRET = "<DOLBYIO_CONSUMER_SECRET>"

# Create a function that will generate a new api access token when needed
async def gen_token():
    response = await authentication.get_api_access_token(CONSUMER_KEY, CONSUMER_SECRET)
    return response['access_token']

print(f"Access Token: {await gen_token()}")

## Step 2: Get the Conference ID

In [ ]:
from dolbyio_rest_apis.communications.monitor import conferences

response = await conferences.get_conferences(await gen_token())
# Save the most recent conference. Change '-1' to whichever conference you want.
confId = response['conferences'][-1]['confId']
print(confId)

## Step 3: Download the Recording

In [ ]:
from dolbyio_rest_apis.communications.monitor import recordings

# Save only the mp3 file and return as a URL.
# If your conference does not use Dolby Voice, use 'download_mp3_recording' instead.
# https://github.com/dolbyio-samples/dolbyio-rest-apis-client-python/blob/main/client/src/dolbyio_rest_apis/communications/monitor/recordings.py
response = await recordings.get_dolby_voice_recordings(await gen_token(), confId)
recording_url = response['url']
print(recording_url)

## Step 4: Send to Deepgram

In [ ]:
from deepgram import Deepgram

# Your Deepgram API Key
DEEPGRAM_API_KEY = '<DEEPGRAM_API_KEY>'

# Location of the file you want to transcribe. Should include filename and extension.
FILE = recording_url

async def main():

  # Initialize the Deepgram SDK
  deepgram = Deepgram(DEEPGRAM_API_KEY)
  
  # file is remote
  # Set the source
  source = {
    'url': FILE
  }

  # Send the audio to Deepgram and get the response
  response = await asyncio.create_task(
    deepgram.transcription.prerecorded(
      source, 
      {
        'punctuate': True
      }
    )
  )

  # Write only the transcript to the console
  print(response['results']['channels'][0]['alternatives'][0]['transcript'])

try:
  await main()
  # If not running in a Jupyter notebook, run main with this line instead:
  # asyncio.run(main())
except Exception as e:
  exception_type, exception_object, exception_traceback = sys.exc_info()
  line_number = exception_traceback.tb_lineno
  print(f'line {line_number}: {exception_type} - {e}')
